# Super deep-daze
## deep-daze & Super-Resolution GAN

## https://github.com/lucidrains/deep-daze

## https://github.com/krasserm/super-resolution

# deep-daze

**Restart after running this cell!**

You must run this cell and then restart and rerun everything for the PyTorch version to be correct. Otherwise the model will run but not produce any meaningful output.

In [ ]:
!nvidia-smi

import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex
!pip install deep-daze --upgrade

In [ ]:
from tqdm import trange
from IPython.display import Image, display
import os
from deep_daze import Imagine

TEXT = "Super Deep Daze" #@param {type:"string"}
NUM_LAYERS = 42 #@param {type:"number"}
SAVE_EVERY =  10#@param {type:"number"}
IMAGE_WIDTH = 512 #@param {type:"number"}
SAVE_PROGRESS = True #@param {type:"boolean"}
LEARNING_RATE = 1e-5 #@param {type:"number"}
ITERATIONS = 10000 #@param {type:"number"}

os.chdir('/content/')

model = Imagine(
                text = TEXT,
                num_layers = NUM_LAYERS,
                save_every = SAVE_EVERY,
                image_width = IMAGE_WIDTH,
                batch_size=64,
                gradient_accumulate_every=1,
                lr = LEARNING_RATE,
                iterations = ITERATIONS,
                save_progress = SAVE_PROGRESS,
                save_date_time = True).cuda()

for epoch in trange(20, desc = 'epochs'):
    for i in trange(ITERATIONS, desc = 'iteration'):
        model.train_step(epoch, i)

        if i % model.save_every != 0:
            continue

        filename = TEXT.replace(' ', '_')
        image = Image(filename + '.png')
        display(image)

# Super-Resolution GAN

In [ ]:
%cd /content/
!git clone https://github.com/krasserm/super-resolution
%cd /content/super-resolution/

In [ ]:
import os
import matplotlib.pyplot as plt

from data import DIV2K
from model.srgan import generator, discriminator

from model import resolve_single
from utils import load_image

%matplotlib inline

In [ ]:
# Location of model weights (needed for demo)
weights_dir = 'weights/srgan'
weights_file = lambda filename: os.path.join(weights_dir, filename)

os.makedirs(weights_dir, exist_ok=True)

In [ ]:
%cd /content/super-resolution/
!wget https://martin-krasser.de/sisr/weights-srgan.tar.gz
!tar -xpf weights-srgan.tar.gz
%cd /content/super-resolution

In [ ]:
gan_generator = generator()
gan_generator.load_weights(weights_file('gan_generator.h5'))

lr = load_image('/content/Super_Deep_Daze.jpg')
image = resolve_single(gan_generator, lr)

In [ ]:
plt.figimage(image, resize=True)
#plt.savefig('/content/TEST')
plt.imshow(image)